In [1]:
#imports
import math
import numpy as np
import random
import itertools
from random import choices
import cmath


In [2]:
#constants
GAIN_MATRIX = (
    (3.95370716e-13, 1.40831353e-12, 1.30345221e-13, 1.58616128e-12),
    (1.10454943e-11, 1.08797904e-13, 1.70626457e-11, 1.87910752e-13),
    (5.35349571e-12, 1.75915771e-12, 1.10933550e-11, 2.15002508e-12),
    (9.57092193e-12, 1.14512900e-11, 7.62246884e-13, 1.58625393e-11)
)

SUM_GAIN = (
    2.63652827e-11, 1.47275591e-11, 2.90485928e-11, 1.97866364e-11
)

D=10**6  #bandwidth
N=-174 + 10 * math.log10(D) #noise in dbm

In [3]:
#generating population in dbm
def generate_random_solutions(powers, no_of_users, num_random_solutions):
    return np.random.choice(powers, size=(num_random_solutions, no_of_users))


In [4]:
#usage
powers = [0,2,4]
no_of_users = 4
num_random_solutions = 8
pop_size_dbm = generate_random_solutions(powers, no_of_users, num_random_solutions)
print(pop_size_dbm)

[[2 4 4 4]
 [0 0 2 0]
 [0 2 0 2]
 [0 2 2 0]
 [4 0 2 4]
 [2 2 4 2]
 [2 0 4 4]
 [2 2 2 4]]


In [5]:
# converting population to linear watt
def calculate_linear_trans_power(pop_size_dbm):
    result=(pow(10, -3) * np.power(10.0, np.array(pop_size_dbm, dtype="float") / 10))
    return result


In [6]:
#usage
linear_pop=calculate_linear_trans_power(pop_size_dbm)
print(linear_pop)

[[0.00158489 0.00251189 0.00251189 0.00251189]
 [0.001      0.001      0.00158489 0.001     ]
 [0.001      0.00158489 0.001      0.00158489]
 [0.001      0.00158489 0.00158489 0.001     ]
 [0.00251189 0.001      0.00158489 0.00251189]
 [0.00158489 0.00158489 0.00251189 0.00158489]
 [0.00158489 0.001      0.00251189 0.00251189]
 [0.00158489 0.00158489 0.00158489 0.00251189]]


In [7]:
#usage for converting the noise to linear
linear_noise = calculate_linear_trans_power([N])
print("Linear noise:", linear_noise)

Linear noise: [3.98107171e-15]


In [8]:
def calculate_formula_and_sums(linear_pop, SUM_GAIN, linear_noise):
    a = np.array(linear_pop)
    b = np.array(SUM_GAIN)
    bit_rate_matrix = (D * np.log2(1 + ((a * b) / linear_noise)))
    return bit_rate_matrix

In [9]:
#Usage for calculating the bitrates through big formula
bitrate=calculate_formula_and_sums(linear_pop, SUM_GAIN, linear_noise)
print(bitrate)

[[3523086.17671602 3363516.1582802  4272652.02625749 3753232.64345308]
 [2930294.44975843 2232475.21650449 3651275.96911754 2577774.03041514]
 [2930294.44975843 2778870.2310227  3052533.55366803 3150104.99872635]
 [2930294.44975843 2778870.2310227  3651275.96911754 2577774.03041514]
 [4140399.83799027 2232475.21650449 3651275.96911754 3753232.64345308]
 [3523086.17671602 2778870.2310227  4272652.02625749 3150104.99872635]
 [3523086.17671602 2232475.21650449 4272652.02625749 3753232.64345308]
 [3523086.17671602 2778870.2310227  3651275.96911754 3753232.64345308]]


In [10]:
def row_wise_sums(matrix):
    return [sum(row) for row in matrix]

In [11]:
# USAGE
row_sum_bitrate = row_wise_sums(bitrate)
print("ROW SUMS oOF BITRATE:")
print(*row_sum_bitrate, sep='\n')


ROW SUMS oOF BITRATE:
14912487.0047068
11391819.665795594
11911803.233175501
11938214.680313796
13777383.667065384
13724713.43272256
13781446.06293109
13706465.02030934


In [12]:
def get_sorted_row_sums(matrix):
    # Calculate the sum of each row and store it as a tuple (row_sum, row)
    row_sums = [(sum(row), row) for row in matrix]

    # Sort the row sums in increasing order
    sorted_row_sums = sorted(row_sums, key=lambda x: x[0])

    # Return the sorted row sums
    return [row_sum for row_sum, _ in sorted_row_sums]

In [13]:
sorted_row_sums = get_sorted_row_sums(bitrate)
print("Row sums in increasing order:")
print(*sorted_row_sums, sep='\n')

Row sums in increasing order:
11391819.665795594
11911803.233175501
11938214.680313796
13706465.02030934
13724713.43272256
13777383.667065384
13781446.06293109
14912487.0047068


In [14]:
target_bitrate = sorted_row_sums[0]
print(target_bitrate)

11391819.665795594


In [15]:
# def get_sorted_matrix(sorted_row_sums, matrix):
#     # Create a dictionary with row sums as keys and corresponding rows as values
#     row_sum_to_row = {sum(row): row for row in matrix}

#     # Get the corresponding matrix rows based on the sorted row sums
#     sorted_matrix = [row_sum_to_row[row_sum] for row_sum in sorted_row_sums]

#     return sorted_matrix


In [16]:
# usage
# sorted_matrix = get_sorted_matrix(sorted_row_sums, bitrate)

# print("Corresponding matrix rows:")
# for row in sorted_matrix:
#     print(row)

In [17]:
def select_half_of_sorted_sums(sorted_row_sums):
    half_length = len(sorted_row_sums) // 2
    return sorted_row_sums[:half_length]

half_sorted_row_sums = select_half_of_sorted_sums(sorted_row_sums)

print("Half of the sorted row sums:")
for row_sum in half_sorted_row_sums:
    print(row_sum)

Half of the sorted row sums:
11391819.665795594
11911803.233175501
11938214.680313796
13706465.02030934


In [18]:
initial_population_indices = [i for i, _ in enumerate(pop_size_dbm) if row_wise_sums(bitrate)[i] in half_sorted_row_sums]
initial_population = [pop_size_dbm[i] for i in initial_population_indices]
print("Initial population:")
for solution in initial_population:
    print(solution)


Initial population:
[0 0 2 0]
[0 2 0 2]
[0 2 2 0]
[2 2 2 4]


In [19]:
initial_population_indices = [i for i, _ in enumerate(pop_size_dbm) if row_wise_sums(bitrate)[i] in half_sorted_row_sums]
initial_population = [pop_size_dbm[i] for i in initial_population_indices]
initial_population_bitrates = [row_wise_sums(bitrate)[i] for i in initial_population_indices]

print("Initial population and their total bitrates:")
for solution, bitrate in zip(initial_population, initial_population_bitrates):
    print("Solution:", solution, "Total bitrate:", bitrate)



Initial population and their total bitrates:
Solution: [0 0 2 0] Total bitrate: 11391819.665795594
Solution: [0 2 0 2] Total bitrate: 11911803.233175501
Solution: [0 2 2 0] Total bitrate: 11938214.680313796
Solution: [2 2 2 4] Total bitrate: 13706465.02030934


In [20]:
def single_point_crossover(parent1, parent2):
    # Perform single-point crossover
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = np.hstack((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.hstack((parent2[:crossover_point], parent1[crossover_point:]))
    
    return child1, child2


offspring = []
for i in range(0, len(initial_population), 2):
    parent1 = initial_population[i]
    parent2 = initial_population[i + 1]
    
    child1, child2 = single_point_crossover(parent1, parent2)
    
    offspring.append(child1)
    offspring.append(child2)

print("Initial population:")
for solution in initial_population:
    print(solution)

print("\nOffspring after single-point crossover:")
for child in offspring:
    print(child)


Initial population:
[0 0 2 0]
[0 2 0 2]
[0 2 2 0]
[2 2 2 4]

Offspring after single-point crossover:
[0 2 0 2]
[0 0 2 0]
[0 2 2 4]
[2 2 2 0]


In [25]:
def mutate_one_gene_per_individual(population, powers):
    mutated_population = np.copy(population)

    for i, individual in enumerate(mutated_population):
        # Choose a random gene within the individual
        gene_index = random.randint(0, len(individual) - 1)

        # Store the original power level of the chosen gene
        original_power = individual[gene_index]

        # Mutate the chosen gene with a random power level from the powers list
        individual[gene_index] = random.choice(powers)

        # Print the mutation information
        print(f"Offspring {i + 1}: Gene {gene_index + 1} was mutated from {original_power} to {individual[gene_index]}")

    return mutated_population


In [26]:
mutated_offspring = mutate_one_gene_per_individual(offspring, powers)

print("\nMutated offspring:")
for child in mutated_offspring:
    print(child)


Offspring 1: Gene 4 was mutated from 2 to 4
Offspring 2: Gene 4 was mutated from 0 to 0
Offspring 3: Gene 3 was mutated from 2 to 0
Offspring 4: Gene 4 was mutated from 0 to 0

Mutated offspring:
[0 2 0 4]
[0 0 2 0]
[0 2 0 4]
[2 2 2 0]
